Loading file


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("hotel_booking.csv", low_memory=False)
print("shape:", df.shape)

shape: (119390, 36)


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [4]:
print(df.isna().sum().sort_values(ascending=False).head(20))

company                           112593
agent                              16340
country                              488
children                               4
hotel                                  0
booking_changes                        0
deposit_type                           0
days_in_waiting_list                   0
customer_type                          0
adr                                    0
reserved_room_type                     0
required_car_parking_spaces            0
total_of_special_requests              0
reservation_status                     0
reservation_status_date                0
name                                   0
email                                  0
phone-number                           0
assigned_room_type                     0
previous_bookings_not_canceled         0
dtype: int64


fixing data type

In [6]:
df['children'] = df['children'].fillna(0).astype(int)
df['agent'] = df['agent'].fillna(0).astype(int)
df['company'] = df['company'].fillna(0).astype(int)
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'], format='%Y-%m-%d', errors='coerce')
df.rename(columns={'phone-number': 'phone_number'}, inplace=True)

handling missing values

In [ ]:
df['country'] = df['country'].fillna("Unknown")
#rest we have handled by fixing data types

outlier detection

In [21]:
cols_with_outliers = [
    'lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 
    'adults', 'children', 'babies', 'previous_cancellations', 
    'previous_bookings_not_canceled', 'booking_changes', 'agent', 
    'company', 'days_in_waiting_list', 'adr', 
    'required_car_parking_spaces', 'total_of_special_requests'
]
outlier_summary = {}

for col in cols_with_outliers:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    
    outlier_summary[col] = {
        "Total Rows": df[col].shape[0],
        "Outlier Count": outliers.shape[0],
        "Outlier %": round(outliers.shape[0] / df[col].shape[0] * 100, 2),
        "Min Value": df[col].min(),
        "Max Value": df[col].max(),
        "Lower Bound": round(lower,2),
        "Upper Bound": round(upper,2)
    }

outlier_report = pd.DataFrame(outlier_summary).T
print(outlier_report)


                                Total Rows  Outlier Count  Outlier %  \
lead_time                         119390.0         3005.0       2.52   
stays_in_weekend_nights           119390.0          265.0       0.22   
stays_in_week_nights              119390.0         3354.0       2.81   
adults                            119390.0        29710.0      24.88   
children                          119390.0         8590.0       7.19   
babies                            119390.0          917.0       0.77   
previous_cancellations            119390.0         6484.0       5.43   
previous_bookings_not_canceled    119390.0         3620.0       3.03   
booking_changes                   119390.0        18076.0      15.14   
agent                             119390.0         1254.0       1.05   
company                           119390.0         6797.0       5.69   
days_in_waiting_list              119390.0         3698.0       3.10   
adr                               119390.0         3793.0       

lead time--> max=737 days- no one books the hotel in 2yrs advance-->(Capping) max 1yr


In [ ]:
df["lead_time"] = np.where(df["lead_time"] > 365, 365, df["lead_time"])


stays_in_week_nights / weekend nights (~3%)---Long stays occur (business + tourism)

adult can't be more than 4 in a single room. (not allowed)

In [23]:
df["adults"] = np.where(df["adults"] > 4, 4, df["adults"])

children & babies can't be more than 3 & 2 respectively.

In [24]:
df["children"] = np.where(df["children"] > 3, 3, df["children"])
df["babies"] = np.where(df["babies"] > 2, 2, df["babies"])

we can keep previous_cancellations, previous_bookings_not_canceled & booking_changes as it is because it shows customer behaviour

no need to handle agent & company as it is Ids.

days_in_waiting_list can be max 3 months.

In [25]:
df["days_in_waiting_list"] = np.where(df["days_in_waiting_list"] > 90, 90, df["days_in_waiting_list"])

adr(Average Daily Rate) --> max = 500


In [26]:
df["adr"] = np.where(df["adr"] > 500, 500, df["adr"])
df["adr"] = np.where(df["adr"] < 0, 0, df["adr"])

normally, tourist can require max 2 parking space.

In [27]:
df["required_car_parking_spaces"] = np.where(df["required_car_parking_spaces"] > 2, 2, df["required_car_parking_spaces"])

special request can be 5. keep it.